In [2]:
!pip install --upgrade langchain-groq langchain faiss-cpu tiktoken wikipedia


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 6.3 MB/s eta 0:00:00


In [23]:
import os
os.environ["GROQ_API_KEY"] = "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"
from langchain_groq.chat_models import ChatGroq

# Initialize the Groq client
llm = ChatGroq(
    model="gemma2-9b-it",
    temperature=0.7,
    max_tokens=1024
)


In [14]:
# import os
# os.environ["GROQ_API_KEY"] = "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"


In [24]:
response = llm.predict("Hello, Groq!")
print(response)


Hello! 👋  

How can I help you today? 😊



In [25]:
from langchain.tools import Tool
import math
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

# Calculator Tool
def calculator_tool(expr: str) -> str:
    try:
        return str(eval(expr, {"__builtins__": {}}, {"math": math}))
    except Exception as e:
        return f"Error: {str(e)}"

calc = Tool(
    name="Calculator",
    func=calculator_tool,
    description="Use this to calculate math expressions like '25*4' or 'math.sqrt(16)'"
)

# Wikipedia Tool
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())


In [26]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [27]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=[calc, wiki],
    llm=llm,  # your ChatGroq instance
    agent_type=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)


In [33]:
# Simple test queries
print(agent.run("Hi, who won the FIFA World Cup in 2018?"))




> Entering new AgentExecutor chain...
Thought: I need to look up the winner of the 2018 FIFA World Cup.
Action: wikipedia
Action Input: FIFA World Cup 2018 winner
Observation: Page: 2018 FIFA World Cup
Summary: The 2018 FIFA World Cup was the 21st FIFA World Cup, the quadrennial world championship for national football teams organized by FIFA. It took place in Russia from 14 June to 15 July 2018, after the country was awarded the hosting rights in late 2010. It was the eleventh time the championships had been held in Europe and the first time they were held in Eastern Europe, also featuring one or two venues, depending on definition, located across the boundary with Asia. At an estimated cost of over $14.2 billion, it was the most expensive World Cup ever held until it was surpassed by the 2022 World Cup in Qatar.
The tournament phase involved 32 teams, of which 31 came through qualifying competitions, while the host nation Russia qualified automatically. Of the 32, 20 had also appea

In [34]:
print(agent.run('What is 256*88+5'))



> Entering new AgentExecutor chain...
Thought: I can use the calculator to solve this.
Action: Calculator
Action Input: 256*88+5
Observation: 22533
Thought:Thought: I now know the final answer
Final Answer: 22533  


> Finished chain.
22533


In [36]:
print(agent.run('What is bmw motorad ?'))



> Entering new AgentExecutor chain...
Thought: I need to look up BMW Motorrad on Wikipedia to learn what it is. 
Action: wikipedia
Action Input: BMW Motorrad
Observation: Page: BMW Motorrad
Summary: BMW Motorrad is the motorcycle brand and division of German automotive manufacturer, BMW. It has produced motorcycles since 1923, and achieved record sales for the fifth year in succession in 2015. With a total of 136,963 vehicles sold in 2015, BMW registered a growth of 10.9%  in sales in comparison with 2014.
In May 2011, the 2,000,000th motorcycle produced by BMW Motorrad was an R1200GS.



Page: BMW F900R
Summary: The BMW F900R is a naked motorcycle introduced by BMW Motorrad in 2019. The F-series 850/900 engine also includes the enduro-inspired F850GS, and the F900XR, a Sport touring design.
It is based on the BMW F850GS series project, with which it shares technical solutions and mechanical parts, also adopted by the F900 XR.

Page: BMW R1200GS
Summary: The BMW R1200GS and R1200GS A